# Far-field Beam Simulation: ML Training Sets
## Simons Observatory Large Aperture Telescope
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyGroup

We will simulate the far-field beam of the SO LAT using ray-tracing technique. We will simulate the SO LAT geometry in five scenarios, with a range of surface panel errors. We will then compute the power spectrum of the far-field beams, to inspect how surface panel errors impact the measurement constraints for SO's measurement of the CMB. 

## Aperture Field Ray Trace
In this section we compute the far-field beams at two receiver-feed (rx) positions, with an adjuster offset RMS of $35\mu m$ on the primary and secondary mirrors. First, we define the geometry of the telescope, including scan resolution receiver-feed position. 

In [ ]:
import numpy as np
import sys
sys.path.append('/home/chesmore/Desktop/Code/holosim_paper/package/holosim-ml')
import scipy
import tele_geo as tg
import ap_field as af
import pan_fitting as pf
import pan_mod as pm
import optics_analyze as oa
import tele_geo as tg
import far_field as ff
import ap_fitting as afit
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import matplotlib.font_manager as font_manager
font_manager.fontManager.addfont(
    "/home/chesmore/.local/share/fonts/times-new-roman.ttf"
)
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams["font.size"] = 20
plt.rcParams["image.cmap"] = "magma"
plt.rcParams["axes.unicode_minus"] = False
adj_pos_m1, adj_pos_m2 = pm.get_single_vert_adj_positions()

save = 1

rx_x = np.array([0])
rx_z = np.array([0])
el = np.array([0])
az = np.array([0])

In [ ]:
iters = 1000
plt.figure(figsize=(4, 3))

for ii in range(0, iters):

    if np.mod(ii + 1, 50) == 0:
        print("Building far-field " + str(ii + 1) + "/" + str(iters))
    adj_1 = np.random.randn(77 * 5) * 35
    adj_2 = np.random.randn(69 * 5) * 35
    tele_geo = tg.initialize_telescope_geometry()
    # Define panels on M1 and M2. Here you can define the
    # magnitude of the adjuster offsets on each mirror:
    pan_mod_m2 = pm.panel_model_from_adjuster_offsets(
        2, adj_2, 1 * (ii + 1), (save)
    )  # Panel Model on M2
    pan_mod_m1 = pm.panel_model_from_adjuster_offsets(
        1, adj_1, 1 * (ii + 1), (save)
    )  # Panel Model on M1
    # Define FOV of RX. In other words, define directions of
    # outgoing rays from the RX.
    th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
    ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)
    # Define the path of the rays from the RX to the aperture plane

    rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
    # Measurement 1
    tele_geo.N_scan = 100
    tele_geo.lambda_ = 0.003
    # Set offsets of Receiver Feed (RX):
    tele_geo.rx_x = rx1[0]
    tele_geo.rx_y = rx1[1]
    tele_geo.rx_z = rx1[2]
    tele_geo.el0 += el[0]
    tele_geo.az0 += az[0]
    rxmirror = af.ray_mirror_pts(rx1, tele_geo, th, ph)
    out1 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2, rx1, tele_geo, th, ph, rxmirror
    )

    beam1 = ff.far_field_sim(out1, tele_geo, rx1)
    amp1 = 20 * np.log10(abs(beam1[2, :]) / np.max(abs(beam1[2, :])))
    #     np.savetxt("/data/chesmore/sim_out/rx000/rx_" +str(rx1)+"_"+str(ii+1)+".txt",\
    #                np.c_[np.real(beam1[0,:]),\
    #                      np.real(beam1[1,:]),\
    #                      np.real(beam1[2,:]),\
    #                      np.imag(beam1[2,:])])

    if ii == 0:
        plt.ylabel("el. [arcmin]")
        plt.xlabel(r"az. [arcmin]")
        first = plt.scatter(
            np.real(beam1[0, :]) * 180 / np.pi * 60,
            np.real(beam1[1, :]) * 180 / np.pi * 60,
            c=amp1,
            vmin=-60,
        )
        plt.axis("equal")
        plt.xlim(-40, 40)
        plt.ylim(24 - 40, 24 + 40)
        plt.colorbar(first, label="dB")
        plt.show()

## Pathlength Differences Due to Adjuster Offsets
Here we plot the pathlength differences at the aperture plane due to adjuster offsets on eachh of the mirrors. In order to see the individualpattern of the panel surface errors from each mirror, we build the aperture field with primary mirror surface errors "turned off", or set to zero, then repeated for the secondary mirror. Left: only the primary mirror's adjuster offsets turned on. Middle: only the secondary mirror's adjuster offsets turned on. Right: both primary and secondary mirrors' adjuster offsets turned on. 

In [ ]:
pan_mod_m2_0 = pm.panel_model_from_adjuster_offsets(
    2, adj_2 * 0, 1, 0
)  # Panel Model on M2
pan_mod_m1_0 = pm.panel_model_from_adjuster_offsets(
    1, adj_1 * 0, 1, 0
)  # Panel Model on M1

out_0 = af.aperature_fields_from_panel_model(
        pan_mod_m1_0, pan_mod_m2_0, rx1, tele_geo, th, ph, rxmirror
    )
out_m2 = af.aperature_fields_from_panel_model(
        pan_mod_m1_0, pan_mod_m2, rx1, tele_geo, th, ph, rxmirror
    )
out_m1 = af.aperature_fields_from_panel_model(
        pan_mod_m1, pan_mod_m2_0, rx1, tele_geo, th, ph, rxmirror
    )

matplotlib.rcParams["font.size"] = 20
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(13, 3), sharey=True)
y_cent_m1 = -7.201003729431267
first = axes[0].scatter(
    out_m1[9, :][np.where(out_m1[15, :] != 0)] / 1e3,
    out_m1[10, :][np.where(out_m1[15, :] != 0)] / 1e3 - y_cent_m1,
    c=1e6 * out_m1[15, :][np.where(out_m1[15, :] != 0)] / tele_geo.k
    - (1e6 * out_0[15, :][np.where(out_m1[15, :] != 0)] / tele_geo.k),
    vmin=-100,
    vmax=100,
)
axes[0].axis("equal")
axes[0].set_ylabel("y (m)")
axes[1].scatter(
    out_m2[9, :][np.where(out_m2[15, :] != 0)] / 1e3,
    out_m2[10, :][np.where(out_m2[15, :] != 0)] / 1e3 - y_cent_m1,
    c=1e6 * out_m2[15, :][np.where(out_m2[15, :] != 0)] / tele_geo.k
    - (1e6 * out_0[15, :][np.where(out_m2[15, :] != 0)] / tele_geo.k),
    vmin=-100,
    vmax=100,
)
axes[1].axis("equal")
axes[2].scatter(
    out1[9, :][np.where(out1[15, :] != 0)] / 1e3,
    out1[10, :][np.where(out1[15, :] != 0)] / 1e3 - y_cent_m1,
    c=1e6 * out1[15, :][np.where(out1[15, :] != 0)] / tele_geo.k
    - (1e6 * out_0[15, :][np.where(out1[15, :] != 0)] / tele_geo.k),
    vmin=-100,
    vmax=100,
)
axes[2].axis("equal")
axes[0].set_xlabel("x (m)")
axes[1].set_xlabel("x (m)")
axes[2].set_xlabel("x (m)")

fig.colorbar(first, ax=axes, label="$\mu m$")
plt.show()